# Train PointNet (https://arxiv.org/abs/1612.00593).

This notebook shows you how to use the PreprocessedDataGenerator in order to train PointNet.

The PreprocessedDataGenerator uses preprocessed-data instead of ETL-data. Wheras ETL-data comes mainly as PCD-files, preprocessed-data comes mainly as pointclouds stored as numpy-arrays. We identified PCD-loading as a bottleneck. 

In [ ]:
%reset
import sys
sys.path.insert(0, "..")

import numpy as np
import os
import random

# Get the dataset path.

This snippet shows you how to get the lates preprocessed path.

In [ ]:
from cgmcore.preprocesseddatagenerator import get_dataset_path
dataset_path = "../../data/preprocessed/2018_10_31_14_19_42"
print("Using dataset path", dataset_path)

In [ ]:
lying = ["SAM-GOV-002", "SAM-GOV-045", "SAM-GOV-051", "SAM-GOV-052", "SAM-GOV-063", "SAM-GOV-068", "SAM-GOV-072", "SAM-GOV-082", "SAM-GOV-088", "SAM-GOV-097", "SAM-SNG-052", "SAM-SNG-065", "MH_WHH_0004", "MH_WHH_0028", "MH_WHH_0032", "MH_WHH_0035", "MH_WHH_0042", "MH_WHH_0047", "MH_WHH_0056", "MH_WHH_0152", "MH_WHH_0161", "MH_WHH_0173", "MH_WHH_0187", "MH_WHH_2909", "MH_WHH_2926", "MH_WHH_2930", "MH_WHH_2960", "MH_WHH_2994", "MP_WHH_0031", "MP_WHH_0035", "MP_WHH_0036", "MP_WHH_0037", "MP_WHH_0038", "MP_WHH_0044", "MP_WHH_0048", "MP_WHH_0061", "MP_WHH_0063", "MP_WHH_0101", "MP_WHH_0133", "MP_WHH_0147", "MP_WHH_0150", "MP_WHH_0153", "MP_WHH_0280", "MP_WHH_0282", "MP_WHH_0285", "MP_WHH_0292", "MP_WHH_2608", "MP_WHH_2612", "MP_WHH_2618", "MP_WHH_2619", "MP_WHH_2651", "MP_WHH_2656", "MP_WHH_2667", "MP_WHH_2676", "MP_WHH_2677", "MP_WHH_2696", "MP_WHH_2697", "MP_WHH_2714", "MP_WHH_2716", "MP_WHH_2718", "MP_WHH_2725"]

# Hyperparameters.

In [ ]:
steps_per_epoch = 20
validation_steps = 10
epochs = 15
batch_size = 100
random_seed = 300

# Create data-generator.

The method create_datagenerator_from_parameters is a convencience method. It allows you to instantiate a generator from a specification-dictionary.

In [ ]:
from cgmcore.preprocesseddatagenerator import create_datagenerator_from_parameters

dataset_parameters_pointclouds = {}
dataset_parameters_pointclouds["input_type"] = "pointcloud"
dataset_parameters_pointclouds["output_targets"] = ["height"]
dataset_parameters_pointclouds["random_seed"] = random_seed
dataset_parameters_pointclouds["pointcloud_target_size"] = 1000
dataset_parameters_pointclouds["pointcloud_random_rotation"] = False
dataset_parameters_pointclouds["sequence_length"] = 0
datagenerator_instance_pointclouds = create_datagenerator_from_parameters(dataset_path, dataset_parameters_pointclouds)

# Getting the QR-Codes and do a train-validate-split.

The data-generator is perfectly capable of retrieving all QR-codes from the dataset. This snipped shows how to do so and how to split the QR-codes into two sets: Train and validate.

In [ ]:
# Get the QR-codes.
qrcodes_to_use = datagenerator_instance_pointclouds.qrcodes[0:1500]

qrcodes_standing = []
for qrcode in qrcodes_to_use:
    if qrcode not in lying:
        qrcodes_standing.append(qrcode)

qrcodes_to_use = qrcodes_standing

# Do the split.
random.seed(random_seed)
qrcodes_shuffle = qrcodes_to_use[:]
random.shuffle(qrcodes_shuffle)
split_index = int(0.7 * len(qrcodes_shuffle))
split_index_1 = int(0.9 * len(qrcodes_shuffle))
qrcodes_train = sorted(qrcodes_shuffle[:split_index])
qrcodes_validate = sorted(qrcodes_shuffle[split_index:split_index_1])
qrcodes_test = sorted(qrcodes_shuffle[split_index_1:])
del qrcodes_shuffle
#print("QR-codes for training:\n", "\t".join(qrcodes_train))
#print("QR-codes for validation:\n", "\t".join(qrcodes_validate))

# Creating python generators for training and validation.

Now both QR-codes lists can be used for creating the actual generators. One for training and one for validation.

In [ ]:
# Create python generators.
generator_pointclouds_train = datagenerator_instance_pointclouds.generate(size=batch_size, qrcodes_to_use=qrcodes_train)
generator_pointclouds_validate = datagenerator_instance_pointclouds.generate(size=batch_size, qrcodes_to_use=qrcodes_validate)
generator_pointclouds_test = datagenerator_instance_pointclouds.generate(size=batch_size, qrcodes_to_use=qrcodes_test)

# Using the generator to create data manually.

Of course you can use the generator to create data manually anytime.

In [ ]:
train_x, train_y = next(generator_pointclouds_train)
print("Input-shape:", train_x.shape)
print("Output-shape:", train_y.shape)

# Training-details.

Training-details are a dictionary that gets stored in a file after training. It is supposed to contain information that is valuable. For example data that is relevant for training including the hyper-parameters. Intended to be used when comparing different models.

In [ ]:
training_details = {
    "dataset_path" : dataset_path,
    "qrcodes_train" : qrcodes_train,
    "qrcodes_validate" : qrcodes_validate,
    "steps_per_epoch" : steps_per_epoch,
    "validation_steps" : validation_steps,
    "epochs" : epochs,
    "batch_size" : batch_size,
    "random_seed" : random_seed,
}

# Training PointNet.

The module modelutils contains methods for creating Neural Nets. The following code shows how to instantiate and train PointNet.

In [ ]:
from keras import models, layers
import tensorflow as tf

def create_point_net(input_shape, output_size, hidden_sizes = [512, 256]):
    """
    Creates a PointNet.

    See https://github.com/garyloveavocado/pointnet-keras/blob/master/train_cls.py

    Args:
        input_shape (shape): Input-shape.
        output_size (int): Output-size.

    Returns:
        Model: A model.
    """

    print("input shape: ", input_shape)
    num_points = input_shape[0]

    def mat_mul(A, B):
        result = tf.matmul(A, B)
        return result

    input_points = layers.Input(shape=input_shape)
    x = layers.Conv1D(64, 1, activation='relu',
                      input_shape=input_shape)(input_points)
    #x = layers.Convolution1D(64, 1, activation='relu',
    #                  input_shape=input_shape)(input_points)
    # Convolution1D(nb_filter, filter_length, activation=None, input_dim=None, input_length=None)
    x = layers.BatchNormalization()(x)
    x = layers.Convolution1D(128, 1, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Convolution1D(1024, 1, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=num_points)(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(9, weights=[np.zeros([256, 9]), np.array([1, 0, 0, 0, 1, 0, 0, 0, 1]).astype(np.float32)])(x)
    input_T = layers.Reshape((3, 3))(x)

    # forward net
    g = layers.Lambda(mat_mul, arguments={'B': input_T})(input_points)
    g = layers.Convolution1D(64, 1, input_shape=input_shape, activation='relu')(g)
    g = layers.BatchNormalization()(g)
    g = layers.Convolution1D(64, 1, input_shape=input_shape, activation='relu')(g)
    g = layers.BatchNormalization()(g)

    # feature transform net
    f = layers.Convolution1D(64, 1, activation='relu')(g)
    f = layers.BatchNormalization()(f)
    f = layers.Convolution1D(128, 1, activation='relu')(f)
    f = layers.BatchNormalization()(f)
    f = layers.Convolution1D(1024, 1, activation='relu')(f)
    f = layers.BatchNormalization()(f)
    f = layers.MaxPooling1D(pool_size=num_points)(f)
    f = layers.Dense(512, activation='relu')(f)
    f = layers.BatchNormalization()(f)
    f = layers.Dense(256, activation='relu')(f)
    f = layers.BatchNormalization()(f)
    f = layers.Dense(64 * 64, weights=[np.zeros([256, 64 * 64]), np.eye(64).flatten().astype(np.float32)])(f)
    feature_T = layers.Reshape((64, 64))(f)

    # forward net
    g = layers.Lambda(mat_mul, arguments={'B': feature_T})(g)
    g = layers.Convolution1D(64, 1, activation='relu')(g)
    g = layers.BatchNormalization()(g)
    g = layers.Convolution1D(128, 1, activation='relu')(g)
    g = layers.BatchNormalization()(g)
    g = layers.Convolution1D(1024, 1, activation='relu')(g)
    g = layers.BatchNormalization()(g)

    # global_feature
    global_feature = layers.MaxPooling1D(pool_size=num_points)(g)

    # point_net_cls
    c = global_feature
    for hidden_size in hidden_sizes:
        c = layers.Dense(hidden_size, activation='relu')(c)
        c = layers.BatchNormalization()(c)
        c = layers.Dropout(rate=0.7)(c)
    
    c = layers.Dense(output_size, activation='linear')(c)
    prediction = layers.Flatten()(c)

    model = models.Model(inputs=input_points, outputs=prediction)
    return model

In [ ]:
from cgmcore import modelutils
from keras import optimizers

input_shape = (dataset_parameters_pointclouds["pointcloud_target_size"], 3)
output_size = 1
model_pointnet = create_point_net(input_shape, output_size, hidden_sizes = [64])
model_pointnet.summary()

sgdl = [(0.0001, 1e-6, 0.2, True), (0.0001, 1e-6, 0.4, True),
        (0.0001, 1e-6, 0.6, True), (0.0001, 1e-6, 0.8, True),
        (0.0001, 1e-6, 0.9, True), (0.001, 1e-6, 0.2, True),
        (0.001, 1e-6, 0.4, True), (0.001, 1e-6, 0.6, True), 
        (0.001, 1e-6, 0.8, True), (0.001, 1e-6, 0.9, True),
        (0.01, 1e-6, 0.2, True), (0.01, 1e-6, 0.4, True),
        (0.01, 1e-6, 0.6, True), (0.01, 1e-6, 0.8, True),
        (0.01, 1e-6, 0.9, True), (0.1, 1e-6, 0.2, True),
        (0.1, 1e-6, 0.4, True), (0.1, 1e-6, 0.6, True), 
        (0.1, 1e-6, 0.8, True), (0.1, 1e-6, 0.9, True)]
for l, d, m, n in sgdl:
    sgd = optimizers.SGD(lr=l,decay=d,momentum=m,nesterov=n)
    model_pointnet.compile(
        optimizer=sgd,
        loss="mse",
        metrics=["mae"]
        )
    print("hyperparams:", l, ",", d, ",", m, ",", n)
    history = model_pointnet.fit_generator(
        generator_pointclouds_train,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        validation_data=generator_pointclouds_validate,
        validation_steps=validation_steps
        )
    
    #model_pointnet.predict_generator(generator_pointclouds_test,steps=10)
    #print(model_pointnet.metrics_names)
    #model_pointnet.evaluate_generator(generator_pointclouds_test,steps=100,use_multiprocessing=True)
    
    output_path = "."
    modelutils.save_model_and_history(output_path, model_pointnet, history, training_details, "pointnet-original")


# Saving everything.

This saves the model, its history and the training-details to some output directory. The created artifacts can later be uses in order to compare different models.